In [7]:
from datetime import datetime, timedelta
import calendar
import re

def parse_date_range_from_text(text: str):
    today = datetime.today()
    start = end = None
    
    # 기준 날짜 (자연어 해석)
    if "내일" in text:
        base = today + timedelta(days=1)
    elif "모레" in text:
        base = today + timedelta(days=2)
    elif "어제" in text:
        base = today - timedelta(days=1)
    elif "오늘" in text:
        base = today
    else:
        base = today

    # 1. 명확한 날짜 범위 패턴 (00월 00일 ~ 00월 00일까지)
    date_range_pattern = re.search(r"(\d{1,2})월\s*(\d{1,2})일\s*부터\s*(\d{1,2})월\s*(\d{1,2})일", text)
    days_match = re.search(r"(\d+)\s*일", text)

    if date_range_pattern:
        start_month = int(date_range_pattern.group(1))
        start_day = int(date_range_pattern.group(2))
        end_month = int(date_range_pattern.group(3))
        end_day = int(date_range_pattern.group(4))

        start_year = today.year if start_month >= today.month else today.year + 1
        end_year = today.year if end_month >= today.month else today.year + 1

        start = datetime(start_year, start_month, start_day)
        end = datetime(end_year, end_month, end_day)

        # n일 여행 표현 있으면
        if days_match:
            days = int(days_match.group(1))
            end = start + timedelta(days=days - 1)

    # 2. 일반적인 표현 대응
    elif re.search(r"오늘부터\s*일주일", text):
        start = today
        end = today + timedelta(days=6)

    elif re.search(r"이번\s*주말", text):
        weekday = today.weekday()
        start = today + timedelta(days=(5 - weekday))
        end = start + timedelta(days=1)

    elif re.search(r"다음\s*주말", text):
        weekday = today.weekday()
        start = today + timedelta(days=(5 - weekday + 7))
        end = start + timedelta(days=1)

    elif re.search(r"이번\s*달", text):
        start = today.replace(day=1)
        end = today.replace(day=calendar.monthrange(today.year, today.month)[1])

    elif re.search(r"다음\s*달", text):
        next_month = today.month % 12 + 1
        year = today.year + (1 if today.month == 12 else 0)
        start = datetime(year, next_month, 1)
        end = datetime(year, next_month, calendar.monthrange(year, next_month)[1])

    elif re.search(r"(\d+)\s*일", text) and not start and not end:
        days = int(re.search(r"(\d+)\s*일", text).group(1))
        start = base
        end = base + timedelta(days=days - 1)

    # 특정 월 언급
    elif re.search(r"(\d{1,2})월.*여행", text):
        month_match = re.search(r"(\d{1,2})월", text)
        if month_match:
            month = int(month_match.group(1))
            year = today.year if month >= today.month else today.year + 1
            start = datetime(year, month, 1)
            end_day = calendar.monthrange(year, month)[1]
            end = datetime(year, month, end_day)


    if start and end:
        return start.strftime("%Y%m%d"), end.strftime("%Y%m%d")
    return None, None

In [2]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

# 행사 정보 가져오기

def get_events(start_date, end_date, area_code=None, sigungu_code=None):
    api_key = os.getenv("KTO_API_KEY")  # .env에서 키 가져오기
    url = "https://apis.data.go.kr/B551011/KorService2/searchFestival2?MobileOS=ETC&MobileApp=Tourmate&eventStartDate=20250101&serviceKey=imXXm%2BLh3dotmxheNARbo8u5Xtuxb4XDPdquwpY0vgQFjs53Is7ofsYZLYly4YMKOjjlafy743m4FqBG%2F7ar5w%3D%3D&eventEndDate=20251231"

    params = {
        "serviceKey": api_key,
        "eventStartDate": start_date,
        "eventEndDate": end_date,
        "MobileOS": "ETC",
        "MobileApp": "TourChat",
        "areaCode": area_code,
        "numOfRows": 20,
        "_type": "json"
    }

    response = requests.get(url, params=params)
    data = response.json()
    return data.get("response", {}).get("body", {}).get("items", {}).get("item", [])

In [3]:
# 지역 코드 가져오기
def get_area_codes(region_name, subregion_name=None):
    import requests
    import os

    api_key = os.getenv("KTO_API_KEY")
    base_url = "https://apis.data.go.kr/B551011/KorService2/areaCode2?MobileOS=ETC&MobileApp=Tourmate&serviceKey=imXXm%2BLh3dotmxheNARbo8u5Xtuxb4XDPdquwpY0vgQFjs53Is7ofsYZLYly4YMKOjjlafy743m4FqBG%2F7ar5w%3D%3D"

    params = {
        "serviceKey": api_key,
        "MobileOS": "ETC",
        "MobileApp": "TourChat",
        "_type": "json",
        "numOfRows": 100
    }

    region_code = None
    subregion_code = None

    r = requests.get(base_url, params=params)
    region_list = r.json()["response"]["body"]["items"]["item"]

    for region in region_list:
        if region_name in region["name"]:
            region_code = region["code"]
            break

    # Step 2: 시군구 코드
    if region_code and subregion_name:
        params["areaCode"] = region_code
        r2 = requests.get(base_url, params=params)
        subregion_list = r2.json()["response"]["body"]["items"]["item"]

        for sub in subregion_list:
            if subregion_name in sub["name"]:
                subregion_code = sub["code"]
                break
    

    return region_code, subregion_code

In [4]:
# 날자 필터링
def filter_events_by_date(events, trip_start, trip_end):
    filtered = []
    for event in events:
        evt_start = event.get("eventstartdate")
        evt_end = event.get("eventenddate")

        if not evt_start or not evt_end:
            continue

        # 문자열 → datetime
        evt_start_date = datetime.strptime(evt_start, "%Y%m%d")
        evt_end_date = datetime.strptime(evt_end, "%Y%m%d")
        trip_start_date = datetime.strptime(trip_start, "%Y%m%d")
        trip_end_date = datetime.strptime(trip_end, "%Y%m%d")

        # 겹치는지 확인
        if evt_start_date <= trip_end_date and evt_end_date >= trip_start_date:
            filtered.append(event)

    return filtered

In [5]:
def chatbot_query(text):
    # 날짜 자동 파싱
    start, end = parse_date_range_from_text(text)
    if not start or not end:
        return "날짜를 이해하지 못했어요."

    # 지역명 추출 
    import re
    match = re.search(r"(서울|부산|대구|울산|광주|대전|인천|세종|제주|제주도|경기|강원|충북|충남|충청남도|전북|전라북도|전남|전라남도|경북|경상북도|경남|울릉도|독도|경상남도)(\s*(\S+))?", text)
    if match:
        region = match.group(1)
        subregion = match.group(3)
    
    else:
        return "🚨 입력하신 지역에 대한 정보가 없어요. 다시 입력해 주세요."

    area_code, sigungu_code = get_area_codes(region, subregion)
    area_code, sigungu_code = get_area_codes(region, subregion)
    if not area_code:
        return f"⚠️ '{region}' 지역에 대한 정보가 존재하지 않습니다."

    # 행사 요청
    events = get_events(start, end, area_code=str(area_code), sigungu_code=str(sigungu_code) if sigungu_code else None)

    # 필터링 추가
    events = filter_events_by_date(events, start, end)

    if not events:
        return f"{region} 지역에서 {start} ~ {end} 기간에 열리는 행사가 없어요."

    reply = f"📍 {start}~{end} 기간 내 {region} 행사\n\n"
    for event in events:
        title = event.get("title")
        addr = event.get("addr1", "주소 정보 없음")
        evt_start = event.get("eventstartdate", "")
        evt_end = event.get("eventenddate", "")

        # YYYYMMDD → YYYY.MM.DD로 포맷 변환
        evt_start_fmt = f"{evt_start[:4]}.{evt_start[4:6]}.{evt_start[6:]}" if evt_start else ""
        evt_end_fmt = f"{evt_end[:4]}.{evt_end[4:6]}.{evt_end[6:]}" if evt_end else ""

        reply += f"- {title} ({addr}) \n ⏰ {evt_start_fmt} ~ {evt_end_fmt}\n"

    return reply

In [8]:
print(chatbot_query("내일부터 5일간 제주 여행갈건데 행사 뭐 있어?"))

📍 20250704~20250708 기간 내 제주 행사

- 구팔일 댕댕이 대잔치! (제주특별자치도 제주시 애월읍 천덕로 880-24) 
 ⏰ 2025.03.01 ~ 2025.12.31
- 마노르블랑 수국축제 (제주특별자치도 서귀포시 안덕면 일주서로2100번길 46) 
 ⏰ 2025.05.23 ~ 2025.08.31
- 보롬왓 메밀축제 (제주특별자치도 서귀포시 표선면 번영로 2350-104) 
 ⏰ 2025.06.18 ~ 2025.07.05
- 보롬왓 수국정원 축제 (제주특별자치도 서귀포시 표선면 번영로 2350-104) 
 ⏰ 2025.05.30 ~ 2025.07.13
- 세계유산축전 (제주특별자치도 제주시 조천읍 선교로 569-36) 
 ⏰ 2025.07.04 ~ 2025.10.22
- 여름꽃 & 능소화축제 (제주특별자치도 제주시 판조로 253-6) 
 ⏰ 2025.05.15 ~ 2025.07.15

